# Supermart Grocery Sales Analysis

This notebook performs a comprehensive data analysis of the Supermart Grocery Sales dataset, covering:
1. Data Cleaning
2. Exploratory Data Analysis (Top Cities, Monthly Trends, Categories)
3. Visualizations


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set style for plots
sns.set(style="whitegrid")
%matplotlib inline

In [ ]:
# Load Data
try:
    df = pd.read_csv('Supermart Grocery Sales - Retail Analytics Dataset.csv')
    print("Data loaded successfully.")
except Exception as e:
    print(f"Error loading data: {e}")

df.head()

In [ ]:
# Check for missing values and data types
print("Missing values:\n", df.isnull().sum())
print("\nData types:\n", df.dtypes)

# Convert Order Date to datetime
# Handling potential format issues by coercing errors, though the dataset usually expects standard formats.
if 'Order Date' in df.columns:
    df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')

# Verify the changes
df.info()

## 1. Top 5 Cities by Sales

In [ ]:
# Analyze Top 5 Cities
if 'City' in df.columns and 'Sales' in df.columns:
    top_cities = df.groupby('City')['Sales'].sum().sort_values(ascending=False).head(5)
    print(top_cities)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x=top_cities.index, y=top_cities.values, palette='viridis')
    plt.title('Top 5 Cities by Sales')
    plt.xlabel('City')
    plt.ylabel('Total Sales')
    plt.tight_layout()
    plt.show()
else:
    print("Required columns for City analysis not found.")

## 2. Monthly Sales Trend

In [ ]:
# Analyze Monthly Sales Trend
if 'Order Date' in df.columns and 'Sales' in df.columns:
    # Create a Month column
    df['Month'] = df['Order Date'].dt.to_period('M')
    monthly_sales = df.groupby('Month')['Sales'].sum()
    
    plt.figure(figsize=(12, 6))
    monthly_sales.plot(kind='line', marker='o')
    plt.title('Monthly Sales Trend')
    plt.xlabel('Month')
    plt.ylabel('Total Sales')
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print("Required columns for Monthly Sales analysis not found.")

## 3. Category Performance

In [ ]:
# Analyze Category Performance
if 'Category' in df.columns and 'Sales' in df.columns:
    category_sales = df.groupby('Category')['Sales'].sum().sort_values(ascending=False)
    print(category_sales)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x=category_sales.index, y=category_sales.values, palette='magma')
    plt.title('Sales by Category')
    plt.xlabel('Category')
    plt.ylabel('Total Sales')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("Required columns for Category analysis not found.")

## 4. Machine Learning: Sales Prediction

In [ ]:
# Data Preprocessing for ML
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

# Prepare data for ML
df_ml = df.copy()

# Encode categorical variables
le = LabelEncoder()
categorical_cols = ['Category', 'Sub Category', 'City', 'Region', 'State']
for col in categorical_cols:
    if col in df_ml.columns:
        df_ml[col] = le.fit_transform(df_ml[col].astype(str))

# Extract date features
if 'Order Date' in df_ml.columns:
    df_ml['Order_Year'] = df_ml['Order Date'].dt.year
    df_ml['Order_Month'] = df_ml['Order Date'].dt.month
    df_ml['Order_Day'] = df_ml['Order Date'].dt.day
    df_ml = df_ml.drop('Order Date', axis=1)

# Drop non-numerical/irrelevant columns
drop_cols = ['Order ID', 'Customer Name', 'Month'] # Month column was created earlier
df_ml = df_ml.drop([c for c in drop_cols if c in df_ml.columns], axis=1)

# Handle any remaining non-numeric columns (if any) or missing values
df_ml = df_ml.dropna()

X = df_ml.drop('Sales', axis=1)
y = df_ml['Sales']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Data prepared. Training shape:", X_train.shape)

In [ ]:
# Initialize and train model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict
y_pred = rf_model.predict(X_test)

# Evaluate
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

# Visualization
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Sales')
plt.ylabel('Predicted Sales')
plt.title('Actual vs Predicted Sales (Random Forest)')
plt.show()